# 2. Multivariate LSTM Model using Keras Tuner

In [1]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from functools import reduce
from pandas import read_csv
from matplotlib import pyplot

# Import LSTM libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from keras.preprocessing.timeseries import TimeseriesGenerator
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import tensorflow as tf 
tf.keras.preprocessing.timeseries_dataset_from_array

<function keras.src.utils.timeseries_dataset_utils.timeseries_dataset_from_array(data, targets, sequence_length, sequence_stride=1, sampling_rate=1, batch_size=128, shuffle=False, seed=None, start_index=None, end_index=None)>

In [2]:
# Import all datasets
oil_data_clean = pd.read_csv('cleaned_oil.csv')
ppi_data_clean = pd.read_csv('cleaned_ppi.csv')
unemp_data_clean = pd.read_csv('cleaned_unemp.csv')
lcpi_data_clean = pd.read_csv('cleaned_lagged_cpi.csv')
cpi_data_clean = pd.read_csv('cleaned_cpi.csv')

In [3]:
oil_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
ppi_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
unemp_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
lcpi_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
cpi_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)

In [4]:
cpi_data_clean.fillna(method='bfill')
lcpi_data_clean.fillna(method='bfill')
oil_data_clean.fillna(method='bfill')
ppi_data_clean.fillna(method='bfill')
unemp_data_clean.fillna(method='bfill')

C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_17212\728974245.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cpi_data_clean.fillna(method='bfill')
C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_17212\728974245.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lcpi_data_clean.fillna(method='bfill')
C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_17212\728974245.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_data_clean.fillna(method='bfill')
C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_17212\728974245.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ppi_data_clea

Date  ChangePercentMonth
0    1948-01            0.175000
1    1948-02            0.175000
2    1948-03           -0.042553
3    1948-04           -0.111111
4    1948-05           -0.150000
..       ...                 ...
882  2021-07           -0.065574
883  2021-08           -0.070175
884  2021-09           -0.132075
885  2021-10           -0.065217
886  2021-11           -0.093023

[887 rows x 2 columns]

In [5]:
oil_data_clean.columns = ['Date','Oil']
oil_data_clean.dropna(inplace = True)
ppi_data_clean.columns = ['Date','PPI']
ppi_data_clean.dropna(inplace = True)
unemp_data_clean.columns = ['Date','Unemployment']
unemp_data_clean.dropna(inplace = True)
lcpi_data_clean.columns = ['Date','LaggedCPI']
lcpi_data_clean.dropna(inplace = True)
cpi_data_clean.columns = ['Date','CPI']
cpi_data_clean.dropna(inplace = True)

dataframe_list = [cpi_data_clean, oil_data_clean, ppi_data_clean, unemp_data_clean, lcpi_data_clean]
# Join all datasets together
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='inner'), dataframe_list)
df_merged.set_index('Date', inplace = True)

In [6]:
values = df_merged.values
# specify columns to plot
groups = [0, 1, 2, 3, 4]
i = 1
# plot each column
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups), 1, i)
	pyplot.plot(values[:, group])
	pyplot.title(df_merged.columns[group], y=0.5, loc='right')
	i += 1
pyplot.show()

In [7]:
# Data Preparation for LSTM Model
df_merged

CPI       Oil       PPI  Unemployment  LaggedCPI
Date                                                          
1983-04  0.007150  0.046464  0.000000     -0.074074   0.000000
1983-05  0.006085 -0.012406  0.003984     -0.020000   0.007150
1983-06  0.003024  0.037355  0.001984      0.040816   0.006085
1983-07  0.004020  0.019758  0.002970     -0.078431   0.003024
1983-08  0.003003 -0.012813  0.004936     -0.021277   0.004020
...           ...       ...       ...           ...        ...
2021-07  0.004811  0.006533  0.012888     -0.065574   0.009291
2021-08  0.002066 -0.073698  0.006750     -0.070175   0.004811
2021-09  0.002716  0.095328  0.009695     -0.132075   0.002066
2021-10  0.008308  0.089964  0.020312     -0.065217   0.002716
2021-11  0.004913 -0.194791  0.011736     -0.093023   0.008308

[464 rows x 5 columns]

In [8]:
df_merged.isnull().sum()

CPI             0
Oil             0
PPI             0
Unemployment    0
LaggedCPI       0
dtype: int64

In [9]:
# Splitting the dataset into train and test sets
df_train = df_merged.iloc[:357]
df_test = df_merged.iloc[357:]

In [10]:
# Use MinMaxScaler to normalise are values by scaling down values between a range of -1 to 1
# Instantiate scaler objects
x_scaler = MinMaxScaler(feature_range=(-1, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))

x_train = x_scaler.fit_transform(df_train[['Oil','PPI','Unemployment','LaggedCPI']])
y_train = y_scaler.fit_transform(df_train[['CPI']])

x_test = x_scaler.transform(df_test[['Oil','PPI','Unemployment','LaggedCPI']])
y_test = y_scaler.transform(df_test[['CPI']])

In [11]:
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(357, 1, 4) (357, 1) (107, 1, 4) (107, 1)


In [20]:
pip install keras-tuner



  Using cached kt_legacy-1.0.5-py3-none-any.whl.metadata (221 bytes)
Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from kerastuner.tuners import Hyperband
import matplotlib.pyplot as plt

# Define the hyperparameter space
def build_model(hp):
    model = Sequential()
    
    # Define the LSTM layer with tunable units
    model.add(LSTM(units=hp.Int('units', min_value=64, max_value=256, step=64), 
                   input_shape=(x_train.shape[1], x_train.shape[2])))
    
    # Output layer
    model.add(Dense(1))
    
    # Compile the model with a tunable learning rate
    model.compile(loss='mse', 
                  optimizer=tf.keras.optimizers.Adam(
                      learning_rate=hp.Float('learning_rate', min_value=0.001, max_value=0.1, sampling='log')))
    return model

# Define the Hyperband tuner
tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=500,
    factor=3,
    directory='my_dir',
    project_name='helloworld'
)

# Perform hyperparameter tuning
tuner.search_space_summary()
tuner.search(x_train, y_train, epochs=50, batch_size=1, 
             validation_data=(x_test, y_test), verbose=2, shuffle=False)

# Get the best hyperparameters and model
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# To plot history, you need to retrain the best model on the full training data
history = best_model.fit(x_train, y_train, epochs=500, batch_size=1, 
                         validation_data=(x_test, y_test), verbose=2, shuffle=False)

# Plot the best model's training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


Trial 725 Complete [00h 35m 11s]
val_loss: 0.015002517029643059

Best val_loss So Far: 0.012897820211946964
Total elapsed time: 13h 07m 36s
Epoch 1/500
357/357 - 7s - 20ms/step - loss: 0.0130 - val_loss: 0.0131
Epoch 2/500
357/357 - 2s - 5ms/step - loss: 0.0130 - val_loss: 0.0129
Epoch 3/500
357/357 - 2s - 6ms/step - loss: 0.0130 - val_loss: 0.0130
Epoch 4/500
357/357 - 2s - 6ms/step - loss: 0.0130 - val_loss: 0.0130
Epoch 5/500
357/357 - 2s - 6ms/step - loss: 0.0130 - val_loss: 0.0135
Epoch 6/500
357/357 - 3s - 7ms/step - loss: 0.0130 - val_loss: 0.0130
Epoch 7/500
357/357 - 2s - 5ms/step - loss: 0.0130 - val_loss: 0.0133
Epoch 8/500
357/357 - 2s - 6ms/step - loss: 0.0130 - val_loss: 0.0130
Epoch 9/500
357/357 - 2s - 7ms/step - loss: 0.0130 - val_loss: 0.0135
Epoch 10/500
357/357 - 2s - 7ms/step - loss: 0.0130 - val_loss: 0.0130
Epoch 11/500
357/357 - 2s - 5ms/step - loss: 0.0130 - val_loss: 0.0136
Epoch 12/500
357/357 - 2s - 6ms/step - loss: 0.0130 - val_loss: 0.0131
Epoch 13/500
357

In [35]:
## Testing the Model
from sklearn.metrics import mean_absolute_error, r2_score
# make a prediction
yhat = best_model.predict(x_test)
# invert scaling for our forecast
inv_yhat = y_scaler.inverse_transform(yhat)
# Add our forecast to the test dataframe and plot the results
df_test['Predicted CPI'] = inv_yhat
df_test[['CPI','Predicted CPI']].plot(figsize=(14,5))
# Calculate RMSE
multi_rmse = sqrt(mean_squared_error(df_test['CPI'], df_test['Predicted CPI']))
print('Test RMSE:', multi_rmse)
# Calculate MAE
multi_mae = mean_absolute_error(df_test['CPI'], df_test['Predicted CPI'])
print('Test MAE:', multi_mae)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Test RMSE: 0.004918753646642198
Test MAE: 0.0023033297149860953


C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_17212\3808429377.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Predicted CPI'] = inv_yhat


In [36]:
df_test

CPI       Oil       PPI  Unemployment  LaggedCPI  Predicted CPI
Date                                                                         
2013-01  0.002957  0.061751  0.004963      0.118421  -0.002693       0.003123
2013-02  0.008190 -0.055801  0.008889     -0.047059   0.002957       0.003588
2013-03  0.002615  0.056274 -0.001468     -0.061728   0.008190       0.002523
2013-04 -0.001040 -0.038774 -0.002451     -0.065789   0.002615       0.002163
2013-05  0.001780 -0.015943  0.002948      0.028169  -0.001040       0.000855
...           ...       ...       ...           ...        ...            ...
2021-07  0.004811  0.006533  0.012888     -0.065574   0.009291       0.004537
2021-08  0.002066 -0.073698  0.006750     -0.070175   0.004811       0.003758
2021-09  0.002716  0.095328  0.009695     -0.132075   0.002066       0.005394
2021-10  0.008308  0.089964  0.020312     -0.065217   0.002716       0.010356
2021-11  0.004913 -0.194791  0.011736     -0.093023   0.008308       0.004476

[107 rows x 6 columns]

In [37]:
print('Multivariate Long Short Memory Network RMSE:' , multi_rmse)
print('Multivariate Long Short Memory Network MAE:' , multi_mae)

Multivariate Long Short Memory Network RMSE: 0.004918753646642198
Multivariate Long Short Memory Network MAE: 0.0023033297149860953


In [39]:
best_model.save('kerastunermodel.h5')